In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from config import CLIENT_ID, CLIENT_SECRET
import pandas as pd
import time

In [10]:
client_credentials_manager = SpotifyClientCredentials(client_id = CLIENT_ID, client_secret = CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [11]:
# https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist

def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

playlist = get_playlist_tracks("23ceahnegd3f2nr3pro2d3ef9", "1dlbVCoN4U1JHZXYbe4SWn")

In [12]:
len(playlist)

10000

In [7]:
playlist[0]['track']['popularity']

75

In [ ]:
# tracks =[] 
# results = sp.album_tracks(album['id'])
# tracks.extend(results['items'])
#             while results['next']:
#                 results = sp.next(results)
#                 tracks.extend(results['items'])
#             track_dict = defaultdict(lambda: defaultdict())
#             for track in tracks:
#                 name = track['name']
#                 track_dict[name]['duration'] = track['duration_ms']
#                 track_dict[name]['explicit'] = track['explicit']
#                 track_dict[name]['artist'] = track['artists'][0]['name']
#                 try:
#                     track_dict[name]['song_popularity'] = track['popularity']
#                 except KeyError:
#                     track_dict[name]['song_popularity'] = np.nan
#                     pass
#                 track_dict[name]['album'] = album['name']

In [13]:
track_ids = [song['track']['uri'].split(':')[2] for song in playlist]
track_names = [song['track']['name'] for song in playlist]
artist_ids = [song['track']['artists'][0]['id'] for song in playlist]
artist_names = [song['track']['artists'][0]['name'] for song in playlist]

In [14]:
release_date = [song['track']['album']['release_date'] for song in playlist]
len(release_date)

10000

In [15]:
# Create a DataFrame
songs_df = pd.DataFrame({
   'track_id': track_ids,
    'track_names': track_names,
    'artist_ids': artist_ids,
    'artist_names': artist_names,
    'release_date': release_date})

songs_df

,track_id,track_names,artist_ids,artist_names,release_date
0,3SWqGa1J0M7hSBUDM0KePD,Legend,2q3GG88dVwuQPF4FmySr9I,The Score,2017-10-13
1,3z8h0TU7ReDPLIbEnYhWZb,Bohemian Rhapsody,1dfeR4HaWDbWqFHLkxsg1d,Queen,2018-10-19
2,0quum0TSd0de8jjpILY2xi,Brawl Stars (Menu Remix V5),43iexp0Suf11TFpvR4OdPh,Hatkuvi,2020-07-06
3,1sWeSMifj6Z6kZyI6z3bRc,Warriors,53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,2015-05-18
4,5CQ30WqJwcep0pYcV4AMNc,Stairway to Heaven - Remaster,36QJpDe2go2KgaRleHCDTp,Led Zeppelin,1971-11-08
...,...,...,...,...,...
9995,0q5lnUuDhlogtYCOubNQhQ,The Rise and Fall of Bossanova Section IV,1TkMNnlmVdWmE2DPzKhZmr,P C III,2016-11-01
9996,6JyUeAB6lGSHH2UdIR643f,The Rise and Fall of Bossanova Section V,1TkMNnlmVdWmE2DPzKhZmr,P C III,2016-11-01
9997,0q5lnUuDhlogtYCOubNQhQ,The Rise and Fall of Bossanova Section IV,1TkMNnlmVdWmE2DPzKhZmr,P C III,2016-11-01
9998,0q5lnUuDhlogtYCOubNQhQ,The Rise and Fall of Bossanova Section IV,1TkMNnlmVdWmE2DPzKhZmr,P C III,2016-11-01


In [17]:
sp.audio_features(track_ids[0])

[{'danceability': 0.601,
  'energy': 0.757,
  'key': 2,
  'loudness': -3.341,
  'mode': 1,
  'speechiness': 0.028,
  'acousticness': 0.0139,
  'instrumentalness': 8.85e-05,
  'liveness': 0.0727,
  'valence': 0.462,
  'tempo': 82.001,
  'type': 'audio_features',
  'id': '3SWqGa1J0M7hSBUDM0KePD',
  'uri': 'spotify:track:3SWqGa1J0M7hSBUDM0KePD',
  'track_href': 'https://api.spotify.com/v1/tracks/3SWqGa1J0M7hSBUDM0KePD',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3SWqGa1J0M7hSBUDM0KePD',
  'duration_ms': 189120,
  'time_signature': 4}]

In [18]:
sp.audio_features(track_ids[0])[0]['danceability']

0.601

In [14]:
audio_features = [sp.audio_features(track_id) for track_id in track_ids]
len(audio_features)

10000

In [34]:
audio_features[7898][0]['id']

'0hLq0ZCLW2ghD3O5XMkDPZ'

In [76]:
feature_track_ids = []
danceability= []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_mins = []
time_signature = []

i= -1
for features in audio_features:
    i += 1
    
    try:
        feature_track_ids.append(features[0]['id'])
        danceability.append(features[0]['danceability'])
        energy.append(features[0]['energy'])
        key.append(features[0]['key'])
        loudness.append(features[0]['loudness'])
        mode.append(features[0]['mode'])
        speechiness.append(features[0]['speechiness'])
        acousticness.append(features[0]['acousticness'])
        instrumentalness.append(features[0]['instrumentalness'])
        liveness.append(features[0]['liveness'])
        valence.append(features[0]['valence'])
        tempo.append(features[0]['tempo'])
        duration_mins.append(round(int(features[0]['duration_ms'])/60000,2))
        time_signature.append(features[0]['time_signature'])
        print(f'Record No:{i}')
        
    except TypeError as e:
        print(e)
        print(f'Type error in record no: {i}')
        feature_track_ids.append('NA')
        danceability.append('NA')
        energy.append('NA')
        key.append('NA')
        loudness.append('NA')
        mode.append('NA')
        speechiness.append('NA')
        acousticness.append('NA')
        instrumentalness.append('NA')
        liveness.append('NA')
        valence.append('NA')
        tempo.append('NA')
        duration_mins.append('NA')
        time_signature.append('NA')
        continue

Record No:0
Record No:1
Record No:2
Record No:3
Record No:4
Record No:5
Record No:6
Record No:7
Record No:8
Record No:9
Record No:10
Record No:11
Record No:12
Record No:13
Record No:14
Record No:15
Record No:16
Record No:17
Record No:18
Record No:19
Record No:20
Record No:21
Record No:22
Record No:23
Record No:24
Record No:25
Record No:26
Record No:27
Record No:28
Record No:29
Record No:30
Record No:31
Record No:32
Record No:33
Record No:34
Record No:35
Record No:36
Record No:37
Record No:38
Record No:39
Record No:40
Record No:41
Record No:42
Record No:43
Record No:44
Record No:45
Record No:46
Record No:47
Record No:48
Record No:49
Record No:50
Record No:51
Record No:52
Record No:53
Record No:54
Record No:55
Record No:56
Record No:57
Record No:58
Record No:59
Record No:60
Record No:61
Record No:62
Record No:63
Record No:64
Record No:65
Record No:66
Record No:67
Record No:68
Record No:69
Record No:70
Record No:71
Record No:72
Record No:73
Record No:74
Record No:75
Record No:76
Record No

Record No:1986
Record No:1987
Record No:1988
Record No:1989
Record No:1990
Record No:1991
Record No:1992
Record No:1993
Record No:1994
Record No:1995
Record No:1996
Record No:1997
Record No:1998
Record No:1999
Record No:2000
Record No:2001
Record No:2002
Record No:2003
Record No:2004
Record No:2005
Record No:2006
Record No:2007
Record No:2008
Record No:2009
Record No:2010
Record No:2011
Record No:2012
Record No:2013
Record No:2014
Record No:2015
Record No:2016
Record No:2017
Record No:2018
Record No:2019
Record No:2020
Record No:2021
Record No:2022
Record No:2023
Record No:2024
Record No:2025
Record No:2026
Record No:2027
Record No:2028
Record No:2029
Record No:2030
Record No:2031
Record No:2032
Record No:2033
Record No:2034
Record No:2035
Record No:2036
Record No:2037
Record No:2038
Record No:2039
Record No:2040
Record No:2041
Record No:2042
Record No:2043
Record No:2044
Record No:2045
Record No:2046
Record No:2047
Record No:2048
Record No:2049
Record No:2050
Record No:2051
Record No:

Record No:3985
Record No:3986
Record No:3987
Record No:3988
Record No:3989
Record No:3990
Record No:3991
Record No:3992
Record No:3993
Record No:3994
Record No:3995
Record No:3996
Record No:3997
Record No:3998
Record No:3999
Record No:4000
Record No:4001
Record No:4002
Record No:4003
Record No:4004
Record No:4005
Record No:4006
Record No:4007
Record No:4008
Record No:4009
Record No:4010
Record No:4011
Record No:4012
Record No:4013
Record No:4014
Record No:4015
Record No:4016
Record No:4017
Record No:4018
Record No:4019
Record No:4020
Record No:4021
Record No:4022
Record No:4023
Record No:4024
Record No:4025
Record No:4026
Record No:4027
Record No:4028
Record No:4029
Record No:4030
Record No:4031
Record No:4032
Record No:4033
Record No:4034
Record No:4035
Record No:4036
Record No:4037
Record No:4038
Record No:4039
Record No:4040
Record No:4041
Record No:4042
Record No:4043
Record No:4044
Record No:4045
Record No:4046
Record No:4047
Record No:4048
Record No:4049
Record No:4050
Record No:

Record No:6057
Record No:6058
Record No:6059
Record No:6060
Record No:6061
Record No:6062
Record No:6063
Record No:6064
Record No:6065
Record No:6066
Record No:6067
Record No:6068
Record No:6069
Record No:6070
Record No:6071
Record No:6072
Record No:6073
Record No:6074
Record No:6075
Record No:6076
Record No:6077
Record No:6078
Record No:6079
Record No:6080
Record No:6081
Record No:6082
Record No:6083
Record No:6084
Record No:6085
Record No:6086
Record No:6087
Record No:6088
Record No:6089
Record No:6090
Record No:6091
Record No:6092
Record No:6093
Record No:6094
Record No:6095
Record No:6096
Record No:6097
Record No:6098
Record No:6099
Record No:6100
Record No:6101
Record No:6102
Record No:6103
Record No:6104
Record No:6105
Record No:6106
Record No:6107
Record No:6108
Record No:6109
Record No:6110
Record No:6111
Record No:6112
Record No:6113
Record No:6114
Record No:6115
Record No:6116
Record No:6117
Record No:6118
Record No:6119
Record No:6120
Record No:6121
Record No:6122
Record No:

'NoneType' object is not subscriptable
Type error in record no: 8206
'NoneType' object is not subscriptable
Type error in record no: 8207
'NoneType' object is not subscriptable
Type error in record no: 8208
'NoneType' object is not subscriptable
Type error in record no: 8209
'NoneType' object is not subscriptable
Type error in record no: 8210
'NoneType' object is not subscriptable
Type error in record no: 8211
'NoneType' object is not subscriptable
Type error in record no: 8212
'NoneType' object is not subscriptable
Type error in record no: 8213
'NoneType' object is not subscriptable
Type error in record no: 8214
'NoneType' object is not subscriptable
Type error in record no: 8215
'NoneType' object is not subscriptable
Type error in record no: 8216
'NoneType' object is not subscriptable
Type error in record no: 8217
'NoneType' object is not subscriptable
Type error in record no: 8218
Record No:8219
Record No:8220
Record No:8221
Record No:8222
Record No:8223
Record No:8224
Record No:822

In [77]:
print(len(feature_track_ids))
print(len(danceability))
print(len(energy))
print(len(key))
print(len(loudness))
print(len(mode))
print(len(speechiness))
print(len(acousticness))
print(len(instrumentalness))
print(len(liveness))
print(len(valence))
print(len(tempo))
print(len(duration_mins))
print(len(time_signature))

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000


In [44]:
# Create a DataFrame
songs_full_df = pd.DataFrame({
   'feature_track_id': feature_track_ids,
    'track_id': track_ids,
    'track_names': track_names,
    'artist_ids': artist_ids,
    'artist_names': artist_names,
    'danceability': danceability,
    'energy': energy,
    'key': key,
    'loudness': loudness,
    'mode': mode,
    'speechiness': speechiness,
    'acousticness': acousticness,
    'instrumentalness': instrumentalness,
    'liveness': liveness,
    'valence': valence,
    'tempo': tempo,
    'duration_mins': duration_mins,
    'time_signature': time_signature})
      
songs_description_df

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3SWqGa1J0M7hSBUDM0KePD,0.601,0.757,2,-3.341,1,0.0280,0.013900,0.000088,0.0727,0.462,82.001,3.15,4
1,3z8h0TU7ReDPLIbEnYhWZb,0.390,0.397,0,-9.963,0,0.0513,0.287000,0.000000,0.2070,0.246,144.031,5.92,4
2,0quum0TSd0de8jjpILY2xi,0.522,0.749,5,-9.181,0,0.0278,0.000391,0.872000,0.1400,0.274,119.974,2.74,4
3,1sWeSMifj6Z6kZyI6z3bRc,0.365,0.851,4,-6.238,0,0.0645,0.096400,0.003000,0.2370,0.309,78.086,2.85,4
4,5CQ30WqJwcep0pYcV4AMNc,0.338,0.340,9,-12.049,0,0.0339,0.580000,0.003200,0.1160,0.197,82.433,8.05,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9587,6nsyYPCDUSacgXci9mvWln,0.814,0.527,9,-7.001,0,0.3280,0.179000,0.000512,0.0767,0.540,170.063,2.56,4
9588,1ThmUihH9dF8EV08ku5AXN,0.935,0.552,10,-9.373,0,0.3350,0.111000,0.000000,0.0952,0.615,99.993,2.43,4
9589,2JYIAzEwodd19jrEVONZno,0.788,0.777,2,-4.998,1,0.2400,0.049400,0.000000,0.2300,0.431,97.605,2.05,4
9590,368hMk29q0OhJuRjWL44Fm,0.878,0.414,8,-11.881,1,0.6630,0.013900,0.000004,0.0935,0.139,154.831,1.89,4


In [73]:
songs_df.to_csv('songs_all.csv', index=False)

In [74]:
songs_description_df.to_csv('songs_description.csv', index=False)

In [78]:
# Create a DataFrame
songs_full_df = pd.DataFrame({
   'feature_track_id': feature_track_ids,
    'track_id': track_ids,
    'track_names': track_names,
    'artist_ids': artist_ids,
    'artist_names': artist_names,
    'danceability': danceability,
    'energy': energy,
    'key': key,
    'loudness': loudness,
    'mode': mode,
    'speechiness': speechiness,
    'acousticness': acousticness,
    'instrumentalness': instrumentalness,
    'liveness': liveness,
    'valence': valence,
    'tempo': tempo,
    'duration_mins': duration_mins,
    'time_signature': time_signature})
      
songs_full_df

,feature_track_id,track_id,track_names,artist_ids,artist_names,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3SWqGa1J0M7hSBUDM0KePD,3SWqGa1J0M7hSBUDM0KePD,Legend,2q3GG88dVwuQPF4FmySr9I,The Score,0.601,0.757,2,-3.341,1,0.028,0.0139,0.000088,0.0727,0.462,82.001,3.15,4
1,3z8h0TU7ReDPLIbEnYhWZb,3z8h0TU7ReDPLIbEnYhWZb,Bohemian Rhapsody,1dfeR4HaWDbWqFHLkxsg1d,Queen,0.39,0.397,0,-9.963,0,0.0513,0.287,0,0.207,0.246,144.031,5.92,4
2,0quum0TSd0de8jjpILY2xi,0quum0TSd0de8jjpILY2xi,Brawl Stars (Menu Remix V5),43iexp0Suf11TFpvR4OdPh,Hatkuvi,0.522,0.749,5,-9.181,0,0.0278,0.000391,0.872,0.14,0.274,119.974,2.74,4
3,1sWeSMifj6Z6kZyI6z3bRc,1sWeSMifj6Z6kZyI6z3bRc,Warriors,53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,0.365,0.851,4,-6.238,0,0.0645,0.0964,0.003,0.237,0.309,78.086,2.85,4
4,5CQ30WqJwcep0pYcV4AMNc,5CQ30WqJwcep0pYcV4AMNc,Stairway to Heaven - Remaster,36QJpDe2go2KgaRleHCDTp,Led Zeppelin,0.338,0.34,9,-12.049,0,0.0339,0.58,0.0032,0.116,0.197,82.433,8.05,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NA,0q5lnUuDhlogtYCOubNQhQ,The Rise and Fall of Bossanova Section IV,1TkMNnlmVdWmE2DPzKhZmr,P C III,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9996,NA,6JyUeAB6lGSHH2UdIR643f,The Rise and Fall of Bossanova Section V,1TkMNnlmVdWmE2DPzKhZmr,P C III,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9997,NA,0q5lnUuDhlogtYCOubNQhQ,The Rise and Fall of Bossanova Section IV,1TkMNnlmVdWmE2DPzKhZmr,P C III,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9998,NA,0q5lnUuDhlogtYCOubNQhQ,The Rise and Fall of Bossanova Section IV,1TkMNnlmVdWmE2DPzKhZmr,P C III,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [81]:
songs_all_df = songs_full_df[songs_full_df['feature_track_id'] != "NA"]

In [82]:
songs_all_df

,feature_track_id,track_id,track_names,artist_ids,artist_names,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,3SWqGa1J0M7hSBUDM0KePD,3SWqGa1J0M7hSBUDM0KePD,Legend,2q3GG88dVwuQPF4FmySr9I,The Score,0.601,0.757,2,-3.341,1,0.028,0.0139,0.000088,0.0727,0.462,82.001,3.15,4
1,3z8h0TU7ReDPLIbEnYhWZb,3z8h0TU7ReDPLIbEnYhWZb,Bohemian Rhapsody,1dfeR4HaWDbWqFHLkxsg1d,Queen,0.39,0.397,0,-9.963,0,0.0513,0.287,0,0.207,0.246,144.031,5.92,4
2,0quum0TSd0de8jjpILY2xi,0quum0TSd0de8jjpILY2xi,Brawl Stars (Menu Remix V5),43iexp0Suf11TFpvR4OdPh,Hatkuvi,0.522,0.749,5,-9.181,0,0.0278,0.000391,0.872,0.14,0.274,119.974,2.74,4
3,1sWeSMifj6Z6kZyI6z3bRc,1sWeSMifj6Z6kZyI6z3bRc,Warriors,53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,0.365,0.851,4,-6.238,0,0.0645,0.0964,0.003,0.237,0.309,78.086,2.85,4
4,5CQ30WqJwcep0pYcV4AMNc,5CQ30WqJwcep0pYcV4AMNc,Stairway to Heaven - Remaster,36QJpDe2go2KgaRleHCDTp,Led Zeppelin,0.338,0.34,9,-12.049,0,0.0339,0.58,0.0032,0.116,0.197,82.433,8.05,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9967,6nsyYPCDUSacgXci9mvWln,6nsyYPCDUSacgXci9mvWln,Whip A Tesla,2YOYua8FpudSEiB9s88IgQ,Yung Gravy,0.814,0.527,9,-7.001,0,0.328,0.179,0.000512,0.0767,0.54,170.063,2.56,4
9968,1ThmUihH9dF8EV08ku5AXN,1ThmUihH9dF8EV08ku5AXN,Faucet Failure,2rhFzFmezpnW82MNqEKVry,Ski Mask The Slump God,0.935,0.552,10,-9.373,0,0.335,0.111,0,0.0952,0.615,99.993,2.43,4
9969,2JYIAzEwodd19jrEVONZno,2JYIAzEwodd19jrEVONZno,Fairly Bruh Parents,3eEJnQZUZcQDjJLkBiqQMd,reptilelegit,0.788,0.777,2,-4.998,1,0.24,0.0494,0,0.23,0.431,97.605,2.05,4
9970,368hMk29q0OhJuRjWL44Fm,368hMk29q0OhJuRjWL44Fm,u U U U u U did i mfken stutter?,55kM6YHZNtzPFa6veTc2Zf,swimcoach,0.878,0.414,8,-11.881,1,0.663,0.0139,0.000004,0.0935,0.139,154.831,1.89,4
